In [1]:
import pkg_resources as pk

import os
import platform
import pynq
import numpy as np
import time
import random
from pynq import Overlay, allocate
from pynq.ps import Clocks
import gzip
from pynq import PL
import dataset_loading as dl
from pynq import allocate

In [3]:
from dataset_loading import mnist
trainx, trainy, testx, testy, valx, valy = mnist.load_mnist_data(
    "fpgannlab/data", one_hot=False, download=True)
#trainx = round(trainx / 255 * 1000).astype(np.uint32)
trainx = trainx.astype(np.uint8)
testx = testx.astype(np.uint8)
#testx = round(testx / 255 * 1000).astype(np.uint32)

Looking for Train Imgs
Tar File found in data_dir. Not Downloading again
Looking for Train Labels
Tar File found in data_dir. Not Downloading again
Looking for Test Imgs
Tar File found in data_dir. Not Downloading again
Looking for Test Labels
Tar File found in data_dir. Not Downloading again


In [99]:
hdc = Overlay("bitfiles/design_1.bit")
Clocks.fclk0_mhz = 50.00
print("Clock is set to " + str(Clocks.fclk0_mhz)) 
dma0 = hdc.axi_dma_0
dma_send = dma0.sendchannel
dma1 = hdc.axi_dma_1
dma_recv = dma1.recvchannel

Clock is set to 50.0


In [100]:
dma0.register_map.S2MM_DMACR.Reset = 1
if not dma_send.running:
    dma_send.start()

dma1.register_map.S2MM_DMACR.Reset = 1
if not dma_recv.running:
    dma_recv.start()

data_size = 784
input_buffer = allocate(shape=(data_size,), dtype=np.uint8)
output_buffer = allocate(shape=(1,), dtype=np.uint8)


total = 0
correct = 0

for i,x in enumerate(testx):
    dma0.register_map.S2MM_DMACR.Reset = 1
    if not dma_send.running:
        dma_send.start()

    dma1.register_map.S2MM_DMACR.Reset = 1
    if not dma_recv.running:
        dma_recv.start()
    input_buffer[:] = testx[i].reshape(784,)
    dma_send.transfer(input_buffer)
    dma_recv.transfer(output_buffer)
    total += 1
    correct += output_buffer[0]==testy[i]
   
print(correct/total*100)
print(correct)

del input_buffer, output_buffer

74.83999999999999
7484
